In [1]:
#!pip install geemap retry

In [2]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [3]:
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=62uAF1zWr2dA3uNquSD7VjkQYqh0_WaJRwYWRwcGz8w&tc=BPgOwLbVk-0fPflnIGvxZa7EnQNuJuSO1sZDtz-MyXw&cc=9KEZRSjP9LChfYA7-RSx6peDU8MJjnX6fH5ah8QlmQg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrxrnaJx5Qwq1JzkF4WlYpbtAB8WwdekPI-A6EFTjO9l8i_d7UDHho

Successfully saved authorization token.


In [4]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [5]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
#Co-ordinates for Kigali
region = ee.Geometry.Polygon(
[
    [
          [29.925360,-1.734857],
          [30.287909,-1.734857],
          [30.287909,-2.168028],
          [29.925360,-2.168028]
     ]
 ] ,None, False,)



In [7]:
image = (
    ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')
    .filterBounds(region)
    .filterDate('2021-06-01','2021-07-01')
    .mosaic()
    .visualize(bands=['R', 'G', 'B'] , min=64,max=5454,gamma=1.8)
    .clip(region)
)

### Set parameters

If you want the exported images to have coordinate system, change `format` to `GEO_TIFF`. Otherwise, you can use `png` or `jpg` formats.

In [39]:
params = {
    'count': 100,  # How many image chips to export
    'buffer': 127,  # The buffer distance (m) around each point
    'scale': 100,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': 'tile_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': '.',  # The output directory. Default to the current working directly
}

In [40]:
Map.addLayer(image, {}, "Image")
Map.addLayer(region, {}, "ROI", False)
Map.setCenter(30.01714, -1.98251,10)
Map

Map(bottom=132816.0, center=[-1.98251, 30.01714], controls=(WidgetControl(options=['position', 'transparent_bg…

In [41]:
def getRequests():
    img = ee.Image(1).rename("Class").addBands(image)
    points = img.stratifiedSample(
        numPoints=params['count'],
        region=region,
        scale=params['scale'],
        seed=params['seed'],
        geometries=True,
    )
    Map.data = points
    return points.aggregate_array('.geo').getInfo()

Function to Download images

In [42]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    point = ee.Geometry.Point(point['coordinates'])
    region = point.buffer(params['buffer']).bounds()

    if params['format'] in ['png', 'jpg']:
        url = image.getThumbURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )

    if params['format'] == "GEO_TIFF":
        ext = 'tif'
    else:
        ext = params['format']

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params['out_dir'])
    basename = str(index).zfill(len(str(params['count'])))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

Downloading images

In [43]:
%%time
logging.basicConfig()
items = getRequests()

pool = multiprocessing.Pool(params['processes'])
pool.starmap(getResult, enumerate(items))

pool.close()

Done:  001
Done:  025
Done:  000
Done:  Done: 009
 Done:  Done: 020 012

016
Done: Done: Done: Done:  Done:   014013
008 Done: Done: 
 Done: Done:  019007  
Done: 005004024

 Done:  
Done: Done: Done: 011  Done:  
 Done: 
006Done: 
002023015  
021

Done: Done:  010
018Done: 
  
017
 Done: 022003
026 


027
Done:  028
Done: Done:  032 
031
Done:  029
Done: Done:   Done: Done: Done: Done: 033  
 Done:   030045Done: 041037036040 




051
Done: 
 039
Done:  038
Done: Done:   053044Done: Done: Done: 
Done: 
   050Done: 042043Done:  Done: Done:  
  Done: Done: 046
Done: 
048
  052 035
034
049

054 
047

Done: Done:  Done:   Done: 057055
063
 
058
Done: Done: Done: Done:  Done:   Done:  064 062061
068

 056Done: 067Done: Done:  


 059 
065060Done: 

Done: Done:    074066075


Done:  077
Done:  Done: 070
 Done: Done: Done:   069 Done: 076082Done: Done: Done: 

    Done: 080
Done: 073 
078072083

 
079
081

Done:  071
Done:  Done: 084 
088
Done:  Done: Done: 087  
085091
Done: Done: Done:   09

In [38]:
#Map.addLayer(Map.data, {}, "Sample points")
#Map